# Projektarbete med LLM's
- Gruppen valde att göra en Sentiment Analysis av IMDB reviews
- Detta är en sandbox för att komma igång litegrann

## Huggingface
- har pretrained models, i sitt library som heter transformers, installera med:  
`pip install transformers tf-keras tensorflow ipykernel ipywidgets`

## Python installation:
- använd version 3.12 (ej 3.13 för den funkar ej med TensorFlow)

## Libraries:
- ipywidgets tf-keras transformers

In [2]:
from transformers import pipeline

In [3]:
# Load the sentiment analysis pipeline with a DistilBERT model
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", framework="tf")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use 0


In [4]:
# Test the pipeline with some sample sentences
sentences = [
    "I love using DistilBERT! It's so fast and accurate.",
    "This product is terrible. I'm very disappointed.",
    "This is very good, not!",
    "Things are going well.",
    "Here in Brazil the sun is shining and the weather is hot.",
    "Here in Sweden the weather is cold and the snow is falling.",
    "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",
    "A wonderful little production. The filming technique is very unassuming - very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only 'has got all the polari' but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done."

]
# Get predictions
results = sentiment_pipeline(sentences)
# Print the results
for sentence, result in zip(sentences, results):
    print(f"Sentence: '{sentence}' => Sentiment: {result['label']}, Confidence: {result['score']:.2f}")

Sentence: 'I love using DistilBERT! It's so fast and accurate.' => Sentiment: POSITIVE, Confidence: 1.00
Sentence: 'This product is terrible. I'm very disappointed.' => Sentiment: NEGATIVE, Confidence: 1.00
Sentence: 'This is very good, not!' => Sentiment: POSITIVE, Confidence: 1.00
Sentence: 'Things are going well.' => Sentiment: POSITIVE, Confidence: 1.00
Sentence: 'Here in Brazil the sun is shining and the weather is hot.' => Sentiment: POSITIVE, Confidence: 1.00
Sentence: 'Here in Sweden the weather is cold and the snow is falling.' => Sentiment: NEGATIVE, Confidence: 0.99
Sentence: 'One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drug